In [9]:
import sys
sys.path.append("../")
from matplotlib import pyplot as plt
from matplotlib.axes import Axes
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from axessettings import AxesSetting, linear_sim
import regex as re
import scipy.constants as const

$E = h\nu = \frac{hc}{\lambda}$

In [10]:
def read_line(str: str) -> list:
    match = re.search("(\d+): (\d+.?\d+)", str)
    return [int(match.group(1)), float(match.group(2))]

def get_spectrums(filename: str) -> pd.DataFrame:
    with open(filename, "r") as file:
        lines = file.readlines()
    nd = np.array([[0, 0]]+[read_line(line) for line in lines])
    df = pd.DataFrame(data = nd[:, 1], index = nd[:, 0], columns=["E"])
    df.index = df.index.astype(int)
    return df

def get_wavelength(E: float):
    return const.h * const.c / (E * const.electron_volt) * 1e9

def get_wl_excite(data, ntoindex, n1, n2):
    return get_wavelength(data.loc[ntoindex(n2)]["E"]-data.loc[ntoindex(n1)]["E"])

def get_wl_excite_simple(data, i1, i2):
    return get_wavelength(data.loc[i2]["E"]-data.loc[i1]["E"])

def ntoi_simple(n):
    return sum([i**2 for i in range(1,n)])

In [11]:
df_H = get_spectrums("pc-spectrum/H.dat")
print(get_wl_excite(df_H, ntoi_simple, 2, 3))
print(get_wl_excite(df_H, ntoi_simple, 2, 4))
print(get_wl_excite(df_H, ntoi_simple, 2, 5))
print(get_wl_excite(df_H, ntoi_simple, 2, 6))

656.139915501695
486.0791093942852
433.87527447228535
409.567251695297


In [25]:
df_He_singlet = get_spectrums("pc-spectrum/He_singlet.dat")
f_He_triplet = get_spectrums("pc-spectrum/He_triplet.dat")
dict_singlet = {
    "1s": 0,
    "2s": 1,
    "2p": 2,
    "3s": 5,
    "3d": 6,
    "3p": 11,
    "4s": 14,
    "4f": 15,
    "4p": 22
}
dict_triplet = {
    "2s": 0,
    "2p": 1,
    "3s": 4,
    "3p": 5,
    "3d": 8,
    "4s": 13,
    "4p": 14,
    "4f": 17,
    "4d": 24
}

print("SINGLET\n---------")
items = list(dict_singlet.items())
for i in range(1, len(items)):
    for j in range(i+1, len(items)):
        wl = get_wl_excite_simple(df_He_singlet, items[i][1], items[j][1])
        if (wl < 1000):
            print(f"{items[i][0]}-{items[j][0]}: {wl: .1f}")

print("\nTRIPLET\n---------")
items = list(dict_triplet.items())
for i in range(len(items)):
    for j in range(i+1, len(items)):
        wl = get_wl_excite_simple(f_He_triplet, items[i][1], items[j][1])
        if (wl < 1000):
            print(f"{items[i][0]}-{items[j][0]}: {wl: .1f}")

SINGLET
---------
2s-3s:  535.9
2s-3d:  503.4
2s-3p:  499.9
2s-4s:  402.4
2s-4f:  396.5
2s-4p:  396.0
2p-3s:  725.8
2p-3d:  667.6
2p-3p:  661.3
2p-4s:  500.8
2p-4f:  491.6
2p-4p:  491.0

TRIPLET
---------
2s-3s:  430.0
2s-3p:  390.2
2s-3d:  383.7
2s-4s:  328.7
2s-4p:  319.0
2s-4f:  318.4
2s-4d:  318.0
2p-3s:  717.6
2p-3p:  613.3
2p-3d:  597.5
2p-4s:  473.9
2p-4p:  454.0
2p-4f:  452.7
2p-4d:  452.1
